In [1]:
import json
import re
import time
from datetime import datetime

In [2]:
from common import close
from config import load_search_time, load_config, input_config
from cookie import load_saved_cookie, load_cookie_from_chrome
from request import find_vaccine
from user import check_user_info_loaded

got_cookie, cookie = load_saved_cookie()
if not got_cookie:
    cookie = load_cookie_from_chrome()

ModuleNotFoundError: No module named 'kakao'

In [14]:
import requests
import urllib3

In [15]:
urllib3.disable_warnings()
header_map = {
    "Accept": "application/json, text/plain, */*",
    "Content-Type": "application/json;charset=utf-8",
    "Origin": "https://vaccine-map.kakao.com",
    "Accept-Language": "en-us",
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 14_7 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 KAKAOTALK 9.4.2",
    "Referer": "https://vaccine-map.kakao.com/",
    "Accept-Encoding": "gzip, deflate",
    "Connection": "Keep-Alive",
    "Keep-Alive": "timeout=5, max=1000"
}

headers_vaccine = {
    "Accept": "application/json, text/plain, */*",
    "Content-Type": "application/json;charset=utf-8",
    "Origin": "https://vaccine.kakao.com",
    "Accept-Language": "en-us",
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 14_7 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 KAKAOTALK 9.4.2",
    "Referer": "https://vaccine.kakao.com/",
    "Accept-Encoding": "gzip, deflate",
    "Connection": "Keep-Alive",
    "Keep-Alive": "timeout=5, max=1000"
}


In [16]:
url = 'https://vaccine-map.kakao.com/api/v3/vaccine/left_count_by_coords'
data = {"bottomRight": {"x": 127.0815041195988, "y": 37.48811824855515}, "onlyLeft": True, "order": "count",
            "topLeft": {"x": 127.01128865531011, "y": 37.54427557646206}}
done = False
found = None

In [39]:
while not done:
        try:
            time.sleep(0.2)
            response = requests.post(url, data=json.dumps(data), headers=header_map, verify=False, timeout=5)
            
            try:
                json_data = json.loads(response.text)

                for x in json_data.get("organizations"):

                    if x.get('status') == "AVAILABLE" or x.get('leftCounts') != 0:
                        found = x
                        done = True
                        break

                if not done:
                    pretty_print(json_data)
                    print(datetime.now())

            except json.decoder.JSONDecodeError as decodeerror:
                print("JSONDecodeError : ", decodeerror)
                print("JSON string : ", response.text)
                close()


        except requests.exceptions.Timeout as timeouterror:
            print("Timeout Error : ", timeouterror)

        except requests.exceptions.SSLError as sslerror:
            print("SSL Error : ", sslerror)
            close()


In [40]:
response.text

'{"organizations":[{"orgCode":"12364991","orgName":"서초아이소아청소년과의원","address":"서울 서초구 사임당로 158","x":127.027174744524,"y":37.4918506610427,"status":"AVAILABLE","leftCounts":9},{"orgCode":"11385332","orgName":"하내과의원","address":"서울 성동구 한림말길 31","x":127.01461467904271,"y":37.54114859111104,"status":"AVAILABLE","leftCounts":8},{"orgCode":"11343940","orgName":"장내과의원","address":"서울 서초구 사임당로 181","x":127.02934630001695,"y":37.49305388727239,"status":"AVAILABLE","leftCounts":8},{"orgCode":"11347031","orgName":"신일외과의원","address":"서울 광진구 뚝섬로 511","x":127.06723444620555,"y":37.53575679070494,"status":"AVAILABLE","leftCounts":8},{"orgCode":"12348546","orgName":"센텀메디의원","address":"서울 강남구 선릉로63길 9","x":127.05141309822802,"y":37.4971333117297,"status":"AVAILABLE","leftCounts":8},{"orgCode":"12308846","orgName":"삼성우리내과의원","address":"서울 강남구 일원로 37","x":127.08123326266507,"y":37.490477951884785,"status":"AVAILABLE","leftCounts":8},{"orgCode":"11378492","orgName":"자애산부인과의원","address":"서울 성동구 성덕정길 79","x":12

In [41]:
found

{'orgCode': '12364991',
 'orgName': '서초아이소아청소년과의원',
 'address': '서울 서초구 사임당로 158',
 'x': 127.027174744524,
 'y': 37.4918506610427,
 'status': 'AVAILABLE',
 'leftCounts': 9}

In [21]:
vaccine_type = 'VEN00013'

In [36]:
organization_code = found.get('orgCode')
check_organization_url = f'https://vaccine.kakao.com/api/v3/org/org_code/{organization_code}'
check_organization_response = requests.get(check_organization_url, headers=headers_vaccine, cookies=cookie, verify=False)

check_organization_data = json.loads(check_organization_response.text).get("lefts")
for x in check_organization_data:
    print(x.get('vaccineCode'))
    if x.get('vaccineCode')== vaccine_type :
        print('yes')
        if x.get('leftCount') != 0:
            print(f"{x.get('vaccineName')} 백신을 {x.get('leftCount')}개 발견했습니다.")
            vaccine_found_code = x.get('vaccineCode')
            break
    #else:
        #print(f"{x.get('vaccineName')} 백신이 없습니다.")


VEN00015
VEN00013
yes
VEN00014


In [32]:
x.get('vaccineCode')

'VEN00014'

In [34]:
sys

'VEN00013'

In [ ]:

done = False
found = None
vaccine_found_code = None

while not done:
    try:
        time.sleep(0.2)
        response = requests.post(url, data=json.dumps(data), headers=header_map, verify=False, timeout=5)

        try:
            json_data = json.loads(response.text)

            for x in json_data.get("organizations"):
                if x.get('status') == "AVAILABLE" or x.get('leftCounts') != 0:
                    found = x
                    print(f"{found.get('orgName')} 에서 백신을 {found.get('leftCounts')}개 발견했습니다.")
                    print(f"주소는 : {found.get('address')} 입니다.")
                    organization_code = found.get('orgCode')
                    # 실제 백신 남은수량 확인
                    vaccine_found_code = None

                    check_organization_url = f'https://vaccine.kakao.com/api/v3/org/org_code/{organization_code}'
                    check_organization_response = requests.get(check_organization_url, headers=headers_vaccine, cookies=cookie, verify=False)
                    check_organization_data = json.loads(check_organization_response.text).get("lefts")
                    for y in check_organization_data:
                        if y.get('vaccineCode') == vaccine_type :
                            if y.get('leftCount') != 0:
                                print(f"{y.get('vaccineName')} 백신을 {y.get('leftCount')}개 발견했습니다.")
                                vaccine_found_code = y.get('vaccineCode')
                                print(f"{vaccine_found_code} 으로 예약을 시도합니다.")
                                done = True
                                break
                            else:
                                print(f"{y.get('vaccineName')}의 물량이 없습니다.")
                        else:
                            print(f"{y.get('vaccineName')} 는 원하는 백신이 아닙니다.")

                    if done :
                        break


            if not done:
                #pretty_print(json_data)
                print(datetime.now())
                found = None
        except json.decoder.JSONDecodeError as decodeerror:
            print("JSONDecodeError : ", decodeerror)
            print("JSON string : ", response.text)
            close()


    except requests.exceptions.Timeout as timeouterror:
        print("Timeout Error : ", timeouterror)

    except requests.exceptions.SSLError as sslerror:
        print("SSL Error : ", sslerror)
        close()

